In [0]:
%pip install pyspark

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install py4j


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install mlflow==2.18.0


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install pandas

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install scikit-learn


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Instalar XGBoost en Databricks
%pip install xgboost


  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/95/a4/16490d38b4854a1ce4995f4088bcb701b5057f711e34c95cd6e29792cdde/xgboost-2.1.2-py3-none-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for nvidia-nccl-cu12 from https://files.pythonhosted.org/packages/ed/1f/6482380ec8dcec4894e7503490fc536d846b0d59694acad9cf99f27d0e7d/nvidia_nccl_cu12-2.23.4-py3-none-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/153.9 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/153.9 MB 5.0 MB/s eta 0:00:31
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/153.9 MB 15.5 MB/s eta 0:00:10
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/153.9 MB 35.0 MB/s eta 0:00:05
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/153.9 MB 29.7 MB/s eta 0:00:06
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/153.9 MB 89.1 MB/s eta 0:00:02
   ━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/153.9 MB 207.6 MB/s eta 0:00:01
 

In [0]:
%restart_python

In [0]:
# Cargar la tabla Delta
df = spark.read.format("delta").table("default.cleaned_retail")

# Mostrar las primeras filas para asegurarse de que los datos se cargaron correctamente
df.show(5)

+--------------+----------+-----------+------+---+----------------+--------+--------------+------------+----+-----+
|Transaction_ID|      Date|Customer_ID|Gender|Age|Product_Category|Quantity|Price_per_Unit|Total_Amount|year|month|
+--------------+----------+-----------+------+---+----------------+--------+--------------+------------+----+-----+
|           974|2023-05-03|    CUST974|  Male| 47|          Beauty|       1|            30|          30|2023|    5|
|           155|2023-05-17|    CUST155|  Male| 31|     Electronics|       4|           500|        2000|2023|    5|
|           758|2023-05-12|    CUST758|  Male| 64|        Clothing|       4|            25|         100|2023|    5|
|           414|2023-05-09|    CUST414|  Male| 48|          Beauty|       4|            25|         100|2023|    5|
|           946|2023-05-08|    CUST946|  Male| 62|     Electronics|       4|           500|        2000|2023|    5|
+--------------+----------+-----------+------+---+----------------+-----

In [0]:
df.printSchema()


root
 |-- Transaction_ID: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Product_Category: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Price_per_Unit: string (nullable = true)
 |-- Total_Amount: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



In [0]:
from pyspark.sql.functions import col, array

# Convertir columnas a tipos numéricos
df = df.withColumn("Age", col("Age").cast("double")) \
       .withColumn("Quantity", col("Quantity").cast("double")) \
       .withColumn("Price_per_Unit", col("Price_per_Unit").cast("double")) \
       .withColumn("Total_Amount", col("Total_Amount").cast("double"))

# Mostrar el esquema para confirmar la transformación
df.printSchema()


root
 |-- Transaction_ID: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Product_Category: string (nullable = true)
 |-- Quantity: double (nullable = true)
 |-- Price_per_Unit: double (nullable = true)
 |-- Total_Amount: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



In [0]:
#DIVISON DE DATOS DE ENTRENAMIENTO 

# Se dividen los datos en entrenamiento y prueba (80% / 20%)
train_df, test_df = df.randomSplit([0.8, 0.2], seed=1234)

# Crear la columna de características
train_df = train_df.withColumn(
    "features",
    array(
        col("Age"),
        col("Quantity"),
        col("Price_per_Unit")
    )
)

test_df = test_df.withColumn(
    "features",
    array(
        col("Age"),
        col("Quantity"),
        col("Price_per_Unit")
    )
)

# Seleccionar solo las columnas necesarias
train_df = train_df.select("features", "Total_Amount")
test_df = test_df.select("features", "Total_Amount")

# Mostrar algunas filas para confirmar
train_df.show(5)
test_df.show(5)



+------------------+------------+
|          features|Total_Amount|
+------------------+------------+
| [52.0, 4.0, 50.0]|       200.0|
| [46.0, 1.0, 50.0]|        50.0|
| [46.0, 1.0, 50.0]|        50.0|
|[34.0, 4.0, 500.0]|      2000.0|
|[37.0, 3.0, 500.0]|      1500.0|
+------------------+------------+
only showing top 5 rows

+------------------+------------+
|          features|Total_Amount|
+------------------+------------+
| [52.0, 4.0, 50.0]|       200.0|
|[34.0, 4.0, 500.0]|      2000.0|
| [40.0, 2.0, 30.0]|        60.0|
| [48.0, 2.0, 50.0]|       100.0|
| [48.0, 2.0, 50.0]|       100.0|
+------------------+------------+
only showing top 5 rows



In [0]:
#MODELO REGRESION LINEAL #
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Convertir el DataFrame de PySpark a Pandas
pandas_df = df.select("Age", "Quantity", "Price_per_Unit", "Total_Amount").toPandas()

# Definir las características (X) y la etiqueta (y)
X = pandas_df[["Age", "Quantity", "Price_per_Unit"]]
y = pandas_df["Total_Amount"]

# Dividir los datos en conjunto de entrenamiento y prueba (80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal con scikit-learn
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predecir valores para los datos de prueba
y_pred = lr.predict(X_test)

# Calcular métricas de evaluación
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"R^2 score: {r2}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")


R^2 score: 0.8419524507656664
RMSE: 228.4449396575831
MAE: 186.05075742770637


In [0]:
#MODELO DE xgboost

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Configuración del modelo XGBoost optimizado
xgb_params = {
    "objective": "reg:squarederror",  # Regresión
    "learning_rate": 0.1,  # Tasa de aprendizaje
    "max_depth": 4,  # Profundidad de los árboles (más bajo para menos complejidad)
    "n_estimators": 100,  # Número de árboles (ajustado para menor tiempo)
    "subsample": 0.8,  # Porcentaje de muestras usadas por árbol
    "colsample_bytree": 0.8,  # Porcentaje de características usadas por árbol
    "tree_method": "hist",  # Método de construcción del árbol (optimizado para menos memoria)
    "verbosity": 1,  # Solo muestra errores críticos
}

# Entrenar el modelo
model = xgb.XGBRegressor(**xgb_params)
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2_score = model.score(X_test, y_test)

print(f"SVR R² Score: {r2_score:.4f}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

SVR R² Score: 0.9990
RMSE: 18.209892411383937
MAE: 13.256557779312134


In [0]:
# MODELO RAMDOM FOREST #

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Convertir el DataFrame de PySpark a Pandas
pandas_df = df.select("Age", "Quantity", "Price_per_Unit", "Total_Amount").toPandas()

# Definir características (X) y variable objetivo (y)
X = pandas_df[["Age", "Quantity", "Price_per_Unit"]]
y = pandas_df["Total_Amount"]

# Dividir los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo Random Forest
rf = RandomForestRegressor(
    n_estimators=30,       # Menor número de árboles para reducir la capacidad de memorización
    max_depth=5,           # Reducir la profundidad de los árboles
    min_samples_split=10,  # Aumentar el número mínimo de muestras necesarias para dividir un nodo
    min_samples_leaf=5,    # Aumentar el número mínimo de muestras en un nodo hoja
    random_state=42
)

rf.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf.predict(X_test)

# Evaluar el modelo
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2_score = rf.score(X_test, y_test)

# Resultados
print(f"Random Forest R² Score: {r2_score:.4f}")
print(f"Random Forest RMSE: {rmse:.4f}")
print(f"Random Forest MAE: {mae:.4f}")

Random Forest R² Score: 1.0000
Random Forest RMSE: 0.0000
Random Forest MAE: 0.0000


In [0]:
# MODELO REGRESION RIDGE #

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Crear y entrenar el modelo de regresión Ridge
ridge = Ridge(alpha=1.0)  # El parámetro alpha controla la regularización
ridge.fit(X_train, y_train)

# Realizar predicciones
y_pred = ridge.predict(X_test)

# Evaluar el modelo
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2_score = ridge.score(X_test, y_test)

# Resultados
print(f"Ridge Regression R² Score: {r2_score:.4f}")
print(f"Ridge Regression RMSE: {rmse:.4f}")
print(f"Ridge Regression MAE: {mae:.4f}")


Ridge Regression R² Score: 0.8419
Ridge Regression RMSE: 228.4557
Ridge Regression MAE: 186.0355


In [0]:
#MODELO GRANDIENT BOOSTING#

from sklearn.ensemble import GradientBoostingRegressor

# Crear y entrenar el modelo de Gradient Boosting
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
gb.fit(X_train, y_train)

# Realizar predicciones
y_pred = gb.predict(X_test)

# Evaluar el modelo
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2_score = gb.score(X_test, y_test)

# Resultados
print(f"Gradient Boosting R² Score: {r2_score:.4f}")
print(f"Gradient Boosting RMSE: {rmse:.4f}")
print(f"Gradient Boosting MAE: {mae:.4f}")


Gradient Boosting R² Score: 1.0000
Gradient Boosting RMSE: 1.6878
Gradient Boosting MAE: 1.0337


In [0]:
#MODELO VECTOR REGRESION

from sklearn.svm import SVR

# Crear y entrenar el modelo SVR
svr = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr.fit(X_train, y_train)

# Realizar predicciones
y_pred = svr.predict(X_test)

# Evaluar el modelo
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2_score = svr.score(X_test, y_test)

# Resultados
print(f"SVR R² Score: {r2_score:.4f}")
print(f"SVR RMSE: {rmse:.4f}")
print(f"SVR MAE: {mae:.4f}")


SVR R² Score: 0.9990
SVR RMSE: 17.7470
SVR MAE: 5.0700


In [0]:
from pyspark.sql.functions import col, weekofyear, dayofweek

# Extraer semana y día de la semana
df = df.withColumn('week', weekofyear(col('Date')))
df = df.withColumn('day_of_week', dayofweek(col('Date')))

# Seleccionar las columnas de interés
df_model = df.select(
    "Age", 
    "Quantity", 
    "Price_per_Unit", 
    "Gender", 
    "Product_Category", 
    "Total_Amount", 
    "year", 
    "week",
    "day_of_week"
)

# Convertir a pandas para entrenamiento del modelo
pandas_df = df_model.toPandas()

# Dividir los datos en características (X) y etiqueta (y)
X = pandas_df[["Age", "Quantity", "Price_per_Unit", "Gender", "Product_Category", "week", "day_of_week"]]
y = pandas_df["Total_Amount"]

df.printSchema()

root
 |-- Transaction_ID: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Product_Category: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Price_per_Unit: string (nullable = true)
 |-- Total_Amount: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)



In [0]:
# EVALUACION DE MODELO GRANDIENT BOOSTING PARA CADA CATEGORIA

# Obtener las categorías de productos de manera eficiente
product_categories = [row['Product_Category'] for row in df.select("Product_Category").distinct().collect()]

# Crear un diccionario para almacenar los modelos por tipo de producto
models_by_category = {}

for category in product_categories:
    # Filtrar los datos por cada tipo de producto
    df_category = df.filter(df.Product_Category == category)

    # Convertir a pandas para usar con scikit-learn
    pandas_df = df_category.select("Age", "Quantity", "Price_per_Unit", "Total_Amount").toPandas()

    # Definir características (X) y variable objetivo (y)
    X = pandas_df[["Age", "Quantity", "Price_per_Unit"]]
    y = pandas_df["Total_Amount"]

    # Dividir los datos en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Crear y entrenar el modelo de Gradient Boosting
    gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
    gb.fit(X_train, y_train)

    # Guardar el modelo en el diccionario
    models_by_category[category] = gb

# Realizar predicciones para cada tipo de producto
predictions_by_category = {}

for category, model in models_by_category.items():
    # Filtrar los datos de test para la categoría
    df_category = df.filter(df.Product_Category == category)
    pandas_df = df_category.select("Age", "Quantity", "Price_per_Unit", "Total_Amount").toPandas()

    # Definir características para predicción
    X_test_category = pandas_df[["Age", "Quantity", "Price_per_Unit"]]
    
    # Predecir
    y_pred_category = model.predict(X_test_category)
    
    # Guardar las predicciones
    predictions_by_category[category] = y_pred_category

# Evaluar el rendimiento de cada modelo
for category, model in models_by_category.items():
    # Filtrar los datos de test para la categoría
    df_category = df.filter(df.Product_Category == category)
    pandas_df = df_category.select("Age", "Quantity", "Price_per_Unit", "Total_Amount").toPandas()

    # Definir características y etiquetas verdaderas
    X_test_category = pandas_df[["Age", "Quantity", "Price_per_Unit"]]
    y_test_category = pandas_df["Total_Amount"]

    # Predecir y evaluar el modelo
    y_pred_category = model.predict(X_test_category)
    
    rmse = mean_squared_error(y_test_category, y_pred_category, squared=False)
    mae = mean_absolute_error(y_test_category, y_pred_category)
    r2 = model.score(X_test_category, y_test_category)

    print(f"Evaluación para la categoría {category}:")
    print(f"R²: {r2:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}\n")


Evaluación para la categoría Electronics:
R²: 1.0000
RMSE: 3.9101
MAE: 1.9473

Evaluación para la categoría Clothing:
R²: 1.0000
RMSE: 1.5323
MAE: 1.0250

Evaluación para la categoría Beauty:
R²: 1.0000
RMSE: 1.5005
MAE: 1.1254



In [0]:
# VALIDACION CRUZADA
from sklearn.model_selection import cross_val_score

# Realizar la validación cruzada (5 particiones)
cv_scores = cross_val_score(gb, X, y, cv=5, scoring='neg_mean_squared_error')

# Convertir las puntuaciones negativas a positivas y obtener RMSE
rmse_cv = (-cv_scores.mean())**0.5

print(f"Cross-validated RMSE: {rmse_cv:.4f}")


Cross-validated RMSE: 1.7726
